# Use Tableau API library to fetch inactive Tableau users from Tableau Online & deactivate their profiles from the platform all at once.

 * Developed by: Kiran Kaushal Kopalley
 * Idea: Kiran Kaushal Kopalley
 * Date Implemented: 10/01/2022
 * Last Change Description: 
 * Last Changed on: 

#### 1. Initializing the libraries required for our script.

In [12]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import flatten_dict_column, querying
import pandas as pd

#### 2. Setting up the Configuration which will have all the information related to the Tableau Online connection, this is a parameter which will be consumed by 'TableauServerConnection' constructor

Note: Personal Access Token needs to be created in Tableau Online under settings; this is unique for Tableau Admin

In [13]:
tableau_config = {
    'tableau_online': {
        'server': 'https://10ay.online.tableau.com',
        'api_version': '3.18',
        'personal_access_token_name':'KK Token',
        'personal_access_token_secret':'JVoJ3yxdSlqRbTeqVMwp2w==:M76dgARVjad9t0X4w4SitVY7RSJW3NsS',
        'site_name': 'prmiinsights',
        'site_url': 'prmiinsights'
    }
}

#### 3. Creating a connection variable and signing into PRMI Tableau Online Server

General HTML Status Codes are linked [here](https://www.restapitutorial.com/httpstatuscodes.html)

* The HTTP 200 indicates that the request has succeeded

In [14]:
conn=TableauServerConnection(tableau_config ,env='tableau_online')
response=conn.sign_in()
response

<Response [200]>

#### 4. Read info of all Tableau users within our Online Server, who haven't logged in yet and write into a dataframe

In [15]:
users_df = querying.get_users_dataframe(conn)
users_df = users_df[['fullName', 'email','authSetting', 'siteRole', ]]
#users_df.to_excel('Tableau Users.xlsx')
#users_df = users_df[users_df['lastLogin'].isna()]
users_df.to_excel("Tableau Users.xlsx")

In [31]:
users_df

,fullName,email,id,siteRole,lastLogin
0,Tableau System Account,tol.admin.api.broker.service.usera@tableau.com,e6dcd20e-21be-4973-80fa-4a2201195722,ServerAdministrator,2022-11-21T16:01:48Z
1,mbolleddu@primeres.com,mbolleddu@primeres.com,05c5c6c8-80eb-49cb-b4ac-ea10954e907b,SiteAdministratorCreator,2022-11-21T20:00:03Z
2,kkopalley@primeres.com,kkopalley@primeres.com,5c5ffb75-507f-4a1e-9e57-81453f3538be,SiteAdministratorCreator,2022-11-21T20:23:35Z
3,Alicia Irwin,airwin@primeres.com,3a3ec0eb-5849-4f50-999d-4c10590efd7d,Viewer,2022-11-21T20:16:36Z
4,Mary hatch,mhatch@primeres.com,6f87ab00-4cf5-4f97-913a-49743673a078,Viewer,2022-11-21T18:07:24Z
...,...,...,...,...,...
282,manh.nguyen@primeres.com,manh.nguyen@primeres.com,1fa1cbc0-8e70-480a-9316-111e976dee26,Viewer,NaN
283,sdepalantino@primeres.com,sdepalantino@primeres.com,3e55dbdc-dce6-4a8d-aa67-4a3f85f7282b,Viewer,NaN
284,christopher_frost@frostmortgageusa.com,christopher_frost@frostmortgageusa.com,07907416-9d56-4772-be17-cbae1b4b3d86,Viewer,2022-10-14T21:08:12Z
285,chris_wood@frostmortgageusa.com,chris_wood@frostmortgageusa.com,649351bf-acdc-49fe-9b6d-083e0f1de1bc,Viewer,2022-10-31T13:43:28Z


#### 5. Write the data to a CSV file

In [6]:
users_df.to_csv('Inactive Tableau Users.csv')

#### 6. Read info of all Inactive Users from CSV file into a new dataframe variable

In [19]:
new_user_df = pd.read_csv('Test accounts.csv')

#### 7. Initialize responses list to capture the html handshakes for each record found in CSV file

In [10]:
responses=[]

#### 8. This loop will remove all the users present in the list from the Tableau Online platform

In [12]:
for index, row in new_user_df.iterrows():
    response = conn.remove_user_from_site(user_id=row['id'])
    responses.append(response)

#### 9. Outputs the response of all the handshakes between our code (Client) and Tableau Online (Server)

General HTML Status Codes are linked [here](https://www.restapitutorial.com/httpstatuscodes.html)

The HTTP 204 No Content success status response code indicates that a request has succeeded, but that the client doesn't need to navigate away from its current page.

In [13]:
responses

[<Response [204]>, <Response [204]>]

In [32]:
import pyodbc
server = 'wsdatawarehouseprod.sql.azuresynapse.net'
database = 'dpdatawarehouse'
username = 'ReportingUser'
password = 'DNp^R3VjHr_G6fzM'   
driver= '{ODBC Driver 17 for SQL Server}'


with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        #query = cursor.execute("SELECT * FROM DIM.EmployeeUltiPro WHERE TerminationDate IS NOT NULL")
        query = "SELECT * FROM DIM.EmployeeUltiPro WHERE TerminationDate IS NOT NULL"
        data = pd.read_sql(query, conn)
        #row = cursor.fetchone()
        #while row:
            #print (row)
            #row = cursor.fetchone()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [33]:
data

,UltiproID,LastName,FirstName,MiddleName,PreferredFirstName,FullNameFML,FullNameLFM,PreferredNameFL,CostCenter,Department,...,CountryCode,FLSAStatus,IsOriginator,SupervisorEmployeeNumber,EepEEID,TerminationReason,TerminationType,RowHash,CreatedDate,ModifiedDate
0,5044,McCullagh,Brian,None,None,Brian McCullagh,"McCullagh, Brian",Brian McCullagh,2168,,...,USA,Non-Exempt,1,1208,None,None,None,"b""\x99'?\x14\x01d\xba\xcdOzP!\x13\xb5\x96B\xd1...",2022-08-11 21:50:42.673,2022-08-25 12:37:22.967
1,11014,Veguilla-Dejesus,Jose,None,None,Jose Veguilla-Dejesus,"Veguilla-Dejesus, Jose",Jose Veguilla-Dejesus,4530,,...,USA,Non-Exempt,1,1104,None,None,None,b'R\xfc\x11\x12_;\x93/\x084\xe2X\xfa\xaf\x95_f...,2022-08-11 21:50:42.673,2022-08-25 12:37:22.967
2,5648,Williams,Daniel,None,None,Daniel Williams,"Williams, Daniel",Daniel Williams,2210,,...,USA,None,None,2075,None,None,None,b'\x8du\xf5\xcbb$E\xe8\x0b$\xec\xc0\xc0\xc0\xa...,2022-08-11 21:50:42.673,2022-08-25 12:37:22.967
3,11614,Campbell,Susan,None,None,Susan Campbell,"Campbell, Susan",Susan Campbell,1000,994,...,USA,Exempt,0,6073,None,Performance,Involuntary,b'XI1C\x98\nh\x01\x87\xf0\xba)\x12Q\xf1\x9b\x1...,2022-08-11 21:50:42.673,2022-08-23 12:36:46.800
4,9455,White,Riley,None,None,Riley White,"White, Riley",Riley White,1000,994,...,USA,Non-Exempt,0,3639,None,None,None,b'\xd8\xe4@\x01#\x9b\xb5\xcf@\xf5\xad\x0fX 9f\...,2022-08-11 21:50:42.673,2022-08-25 12:37:22.967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6277,10556,Daniels,Xavier,None,None,Xavier Daniels,"Daniels, Xavier",Xavier Daniels,1000,994,...,USA,Non-Exempt,0,11698,None,Violation of Rules/Reg,Involuntary,"b'K\xf4\x10sn\x84\x1a3,\x1c\x9a\xfc\xd5,\xeb\x...",2022-08-11 21:50:42.673,2022-08-23 12:36:46.800
6278,1931,Nakasone,Wayde,None,None,Wayde Nakasone,"Nakasone, Wayde",Wayde Nakasone,1008,,...,USA,None,1,82,None,None,None,b'\x88\xb3\x0bG\xc7\xa6\x055>\xa0b\xf7eeW\xf7\...,2022-08-11 21:50:42.673,2022-08-25 12:37:22.967
6279,6544,Bauer,Dirk,None,None,Dirk Bauer,"Bauer, Dirk",Dirk Bauer,1000,975,...,USA,Non-Exempt,0,547,None,Abandoned Job,Voluntary,"b""\xdd1\x9b\x9c\xaei\x88T\n\xaf<\xa9q\x82\x1d\...",2022-08-11 21:50:42.673,2022-10-06 12:37:12.887
6280,14872,Gebremichael,DeAnna,None,None,DeAnna Gebremichael,"Gebremichael, DeAnna",DeAnna Gebremichael,1004,,...,USA,Non exempt,0,10755,None,Excessive absence/tardy,Involuntary,b'-\x90\xe95\x0c\x86\x0f\xd6\xa8\xf3\x94\x03\x...,2022-09-07 12:37:01.747,2022-10-07 12:37:24.723


In [36]:
finaldf = pd.merge(users_df, data, left_on='email', right_on='EmailAddress', how='left').drop('EmailAddress', axis=1)

In [37]:
finaldf.to_excel("Tableau Users Curated data.xlsx")

In [ ]:
finaldf[finaldf['TerminationDate'].isna()]

In [38]:
finaldf.loc[finaldf['TerminationDate'].notnull()]

,fullName,email,id,siteRole,lastLogin,UltiproID,LastName,FirstName,MiddleName,PreferredFirstName,...,CountryCode,FLSAStatus,IsOriginator,SupervisorEmployeeNumber,EepEEID,TerminationReason,TerminationType,RowHash,CreatedDate,ModifiedDate
6,tshoaf@primeres.com,tshoaf@primeres.com,23fc416b-d099-437e-8c60-31b8eaa698f1,Viewer,2022-10-10T17:13:51Z,14283,Shoaf,Trey,None,None,...,USA,Exempt,0,3082,None,Lack of work,Involuntary,b'\xa6\xe6\xfa\x9b\xa5\x0f\xe8FE\xcb\x08Zj1\x8...,2022-08-11 21:50:42.673,2022-10-17 12:37:46.353
15,aswope@primeres.com,aswope@primeres.com,6897869b-3175-4e54-ad26-42c1707d6f56,Viewer,2022-10-28T15:30:14Z,2072,Swope,Andrew,None,None,...,USA,Exempt,0,12124,None,Lack of work,Involuntary,b'\xfd3]\x8e\xfb\xff\x92\xebbJ(\x0b\xe0W\xf8S\...,2022-08-11 21:50:42.673,2022-11-04 12:36:56.420
33,grooker@primeres.com,grooker@primeres.com,3005ca6b-2b7f-417a-8e8f-3a7a90a844ee,Viewer,2022-09-20T17:15:19Z,6511,Rooker,Geoff,None,None,...,USA,Exempt,1,137,None,Other job,Voluntary,b'\x01\x00\x1e6\xa2\xaac\xda\xb9\xc6\xb2\x7f\x...,2022-08-11 21:50:42.673,2022-10-18 12:37:29.900
37,jhutelmyer@primeres.com,jhutelmyer@primeres.com,c2b0e9ee-5315-4dac-9a46-7128dad25943,Viewer,NaN,6525,Hutelmyer,James,None,None,...,USA,Exempt,1,137,None,Other job,Voluntary,b'\xc3\xae\xcel\xed\xefqyb)W5.\xfe1\x9d-\x98\x...,2022-08-11 21:50:42.673,2022-10-18 12:37:29.900
58,anorman@primeres.com,anorman@primeres.com,3a48cba4-8397-4f15-b6b9-f1280d159fde,Viewer,NaN,45,Norman,Alandro,D.,None,...,USA,Non-Exempt,1,1247,None,<None>,Voluntary,"b'\xe2\xd7\xbe\ts\x8b\xcd\xbbT\x08\xf0\x07,E\x...",2022-08-11 21:50:42.673,2022-11-11 20:37:01.580
69,bmacht@primeres.com,bmacht@primeres.com,e81fe82b-25c7-4d3b-af85-4aee0fde71de,Viewer,NaN,5983,Macht,Benjamin,None,None,...,USA,Exempt,1,137,None,<None>,Voluntary,b'\xb0\xa5qI\xa9]Y\x88\x9f\xc58wx\x00\x1cv\xc1...,2022-08-11 21:50:42.673,2022-10-17 12:37:46.353
70,bmineau@primeres.com,bmineau@primeres.com,2b99b504-2df8-4491-b208-79c131746b72,Viewer,NaN,12697,Mineau,Brian,None,None,...,USA,Exempt,1,11715,None,Performance,Involuntary,b'\xf2\x14\xa9w\x81\xb0\xabL\xcf9u\xaa\xbfXl\x...,2022-08-11 21:50:42.673,2022-08-23 12:36:46.800
80,christopherr@primeres.com,christopherr@primeres.com,1750fc14-e0e6-4140-b4b6-0bff467f70d4,Viewer,NaN,13061,Roberts,Christopher,None,None,...,USA,Exempt,1,12608,None,Location Closed,Involuntary,"b'\xcb&\x862\x99t\xec\x87l\x82r<\xf5,Mou\xbe\x...",2022-08-11 21:50:42.673,2022-10-20 12:37:10.283
110,ewalters@primeres.com,ewalters@primeres.com,2a2bccbd-3779-4e6b-a4c5-67c84a069191,Viewer,2022-06-08T15:15:32Z,5676,Walters,Edward,None,None,...,USA,Exempt,1,9387,None,Lack of work,Involuntary,b'\x18\xb8\xa8\xd1#{B\xff\xfb~\x84v\xc2\xbb\x8...,2022-08-11 21:50:42.673,2022-10-26 12:37:20.950
115,gkerkbashian@primeres.com,gkerkbashian@primeres.com,380eb151-20b5-4d7e-a0a8-5b3baf106258,Viewer,2022-01-04T22:31:15Z,13986,Kerkbashian,Gil,None,None,...,USA,Exempt,1,137,None,Other job,Voluntary,b'\x9e\xa6zi\xb6\xa4K\x9c+\xfa\xea\nFW\xfc\xe1...,2022-08-11 21:50:42.673,NaT
